# Steam-Methane Reforming

Steam Methane Reforming (SMR) Reaction (800-870C and 20-30 atm):

$CH_4 + H_2O \rightarrow 3H_2 + CO, \Delta H^{\circ} = 225.4$ kJ/mol

Water Gas Shift (WGS) Reaction:

$CO + H_2O \rightarrow H_2 + CO_2, \Delta H^{\circ} = -42$ kJ/mol

<cite data-cite="6356110/MUGW5263"></cite>
https://linkinghub.elsevier.com/retrieve/pii/B9780444641274000148

In [1]:
mh2 = 1  # hydrogen mass [kg]
Mh = 1.008  # hydrogen molar mass [g/mol]
nomolh2 = mh2*1e3/2/Mh  # number of h2 mols
nomolh2

496.031746031746

In [2]:
dG = 225.4  # E[kJ/mol] required to split the H2O molecule
E = dG * nomolh2  # E[kJ] to produce 1kg of H2
E = E/3600  # E [kWh]
E

31.0570987654321

That's the thermal energy required to do the chemical conversion, we need to add the thermal energy to produce the steam.

What's the steam pressure and temperature necessary for the steam methane reforming?
P = 20-30 atm, P$_{ave}$ = 25 atm
T = 900 C

### Mass Balance

In [3]:
mw = 1  # mass of water [kg]
Mw = 18.0153  # water molar mass [g/mol]
nomolw = mw*1e3/Mw  # Number of mols of H_2O

Mc = 16.04  # ch4 molar mass [g/mol]
mc = nomolw*Mc/1e3  # ch4 mass [kg]

print("CH4 mass: ", mc)

nomolh2 = 3*nomolw  # Number of mols of H_2 produced
nomolco = nomolw  # Number of mols of CO produced

nomolh2 = nomolh2 + nomolco  # Number of mols produced after both reactions
mh = nomolh2*Mh/1e3

print("Hydrogen mass: ", mh)

mw = mw + nomolco*Mw/1e3  # mass of water
print("Water mass: ", mw)

CH4 mass:  0.8903543099476555
Hydrogen mass:  0.22380976170255285
Water mass:  2.0


In [4]:
mw/mh #mass of water required for 1kg of Hydrogen

8.936160714285714

# Real numbers:

Air Liquide:
https://www.engineering-airliquide.com/steam-methane-reforming-hydrogen-production

E = 12.3 - 13.2 MJ/Nm$^3$
Steam production = 0.4 - 1.2 kg/Nm3
Capacity = 10e3 - 200e3 Nm3/h (small plants: 10e3 - 40e3 Nm3/h)

In [7]:
E = (12.3+13.2)/2*1e3  # [kJ/Nm3]
E = E/3600/0.084  #kWh/kg
print("Specific energy: ", E, "kWh/kg H2")

Specific energy:  42.16269841269841 kWh/kg H2


In [10]:
w = (0.4+1.2)/2  #kg/Nm3 H2
w = w/0.084
print("Water mass necessary: ", w, "kg/kg H2")

Water mass necessary:  9.523809523809524 kg/kg of H2


In [13]:
cap = 40e3  # Nm3/h
cap = cap*0.084  #kg/h
print(cap, "kg of H2/h")

3360.0 kg of H2/h
